In [40]:
import json
from authorization import SpotifyAuth
import spotify_functions as spotify
import os
import pandas as pd
from google.cloud import bigquery
from helpers.helpers import get_url

In [18]:
import datetime
yesterday = datetime.datetime.now() - datetime.timedelta(days=2)
cdc_time = int(yesterday.timestamp()*1e3)
print(f"Running from time period: {cdc_time}")

Running from time period: 1729254157594


In [19]:
with open('client-secrets-rnr.json','r+') as secrets_file:
    secrets = json.load(secrets_file)

client_id = secrets["client_id"]
client_secret = secrets["client_secret"]
redirect_url = secrets["redirect_url"]

## Check records in the table

In [33]:
credentials_path = 'bigquery_pk.json'

In [34]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
client = bigquery.Client()

In [44]:
def get_new_records (dataframe, bq_client, bq_table, primary_keys):
    """
    Determine the presence of records in the BQ Table.
    Returns WID-populated dataframe to be loaded to table.
    
    Parameters:
    ==========
    dataframe    :    pandas DataFrame
    dataframe containing rows to check for in BQ Table
        
    bq_table     :    BigQuery Table
    BigQuery Table checking primary keys against
    
    primary_keys :    list or str
    List of primary keys to check records presence against
    
    Returns:
    ========
    
        i)  dataframe with WID columns if new records need to be uploaded
        ii) None if all records already exist in BQ Table
    """
    
    # Type casting Primary Keys to list for seamless handling of data types
    if not(isinstance(primary_keys, list)):
        primary_keys = [primary_keys]
        
    
    bq_df = bq_client.query(f"SELECT * FROM `{bq_table}`").to_dataframe()
    
    df_diff = dataframe.merge(bq_df.drop_duplicates(), on=primary_keys, how='left', indicator=True)
    
    return df_diff

In [45]:
spotify_authorisation = SpotifyAuth(client_id=client_id, client_secret=client_secret, redirect_url=redirect_url)
spotify_authorisation.set_access_token_from_file('access_token.json')

user = [spotify.get_user_details(spotify_authorisation.get_access_token())]
df_user = pd.DataFrame(user)
df_user['spotify_url'] = df_user['external_urls'].apply(lambda x:x['spotify'])
df_user['profile_pic'] = df_user['images'].apply(get_url, args=[300])
df_user = df_user[['display_name','type','country','product','spotify_url','profile_pic']]
df_user

,display_name,type,country,product,spotify_url,profile_pic
0,sdspot2034,user,IN,premium,https://open.spotify.com/user/sdspot2034,https://i.scdn.co/image/ab6775700000ee8580153a...


In [46]:
get_new_records(df_user, client, bigquery.Table("rhyme-and-reason.Spotify_db.DIM_USER"),['spotify_url'])

/Users/shreyan/opt/anaconda3/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2309: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/Users/shreyan/opt/anaconda3/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2323: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/Users/shreyan/opt/anaconda3/lib/python3.9/site-packages/google/cloud/bigquery/table.py:2337: UserWarning: Unable to represent RANGE schema as struct using pandas ArrowDtype. Using `object` instead. To use ArrowDtype, use pandas >= 1.5 and pyarrow >= 10.0.1.
  warnings.warn(_RANGE_PYARROW_WARNING)
/Users/shreyan/opt/anaconda3/lib/python3.9/site-packages/google/cloud/bigquery/table.py:1727: UserWarni

,display_name_x,type_x,country_x,product_x,spotify_url,profile_pic_x,user_wid,display_name_y,type_y,country_y,product_y,profile_pic_y,_merge
0,sdspot2034,user,IN,premium,https://open.spotify.com/user/sdspot2034,https://i.scdn.co/image/ab6775700000ee8580153a...,1,sdspot2034,user,IN,premium,https://i.scdn.co/image/ab6775700000ee8580153a...,both
